
# Capítulo 2 · Fase 1 — Filtrado de contexto e idioma

Este notebook carga un dataset de tickets/consultas y aplica el primer filtro:
- **Contexto**: mantener solo `tipo` ∈ {`soporte`, `producto`}
- **Idioma**: mantener solo `idioma = 'es'`

Archivos de entrada/salida:
- Entrada: `tickets_soporte.xlsx`
- Salida (Fase 1): `dataset_fase1.xlsx`


In [ ]:

import pandas as pd
from pathlib import Path

in_path = Path('tickets_soporte.xlsx')  # asegúrate de tenerlo en la misma carpeta
df = pd.read_excel(in_path)

print('Shape crudo:', df.shape)
display(df.head())
print('\nDistribución por tipo:')
display(df['tipo'].value_counts())
print('\nDistribución por idioma:')
display(df['idioma'].value_counts())


In [ ]:

# Reglas de Fase 1
allowed_types = {'soporte', 'producto'}
allowed_langs = {'es'}

before = len(df)
df_f1 = df[df['tipo'].isin(allowed_types) & df['idioma'].isin(allowed_langs)].copy()
removed = before - len(df_f1)

print(f'Registros iniciales: {before}')
print(f'Registros eliminados (fuera de contexto/idioma): {removed}')
print(f'Registros tras Fase 1: {len(df_f1)}')
display(df_f1.head())


In [ ]:

out_path = Path('dataset_fase1.xlsx')
df_f1.to_excel(out_path, index=False)
print('Guardado:', out_path.resolve())



> **Siguiente (Fase 2 – Limpieza):** eliminar filas sin texto, rellenar no críticos con `"desconocido"` y quitar duplicados.


In [ ]:
# Fase 2 - Limpieza de datos
# En esta fase, se realizarán las siguientes tareas:
# 1. Eliminar filas sin texto.
# 2. Rellenar valores no críticos con "desconocido".
# 3. Eliminar duplicados.

In [ ]:
# Eliminar filas sin texto
df_f2 = df_f1.dropna(subset=['texto']).copy()

# Rellenar valores no críticos con "desconocido"
columns_to_fill = ['canal', 'plataforma', 'versión']
df_f2[columns_to_fill] = df_f2[columns_to_fill].fillna('desconocido')

# Eliminar duplicados
before_dupes = len(df_f2)
df_f2 = df_f2.drop_duplicates().copy()
removed_dupes = before_dupes - len(df_f2)

print(f'Registros iniciales (Fase 1): {len(df_f1)}')
print(f'Registros eliminados (sin texto): {len(df_f1) - len(df_f2) + removed_dupes}')
print(f'Registros eliminados (duplicados): {removed_dupes}')
print(f'Registros tras Fase 2: {len(df_f2)}')
display(df_f2.head())

In [ ]:
# No se rellenan campos no críticos ya que se hizo en la fase anterior (Fase 2)
# Mostrar los primeros registros para verificar
display(df_f2.head())

In [ ]:
display(df_f2.head())

In [ ]:
# Rellenar campos con un valor por defecto en pandas
# Identificar las columnas a rellenar y el valor por defecto
columns_to_fill = ['prioridad', 'urgencia', 'impacto']
default_value = 'desconocido'

# Rellenar los valores NaN en las columnas especificadas con el valor por defecto
df_f2[columns_to_fill] = df_f2[columns_to_fill].fillna(default_value)

# Mostrar los primeros registros para verificar
display(df_f2.head())

In [ ]:
# Eliminar registros duplicados en pandas
# Eliminar duplicados
before_dupes = len(df_f2)
df_f3 = df_f2.drop_duplicates().copy()
removed_dupes = before_dupes - len(df_f3)

print(f'Registros iniciales (Fase 2): {len(df_f2)}')
print(f'Registros eliminados (duplicados): {removed_dupes}')
print(f'Registros tras eliminar duplicados: {len(df_f3)}')
display(df_f3.head())

In [ ]:
# Convertir todo el texto a minúsculas
df_f3['texto'] = df_f3['texto'].str.lower()

# Mostrar los primeros registros para verificar
display(df_f3.head())

In [ ]:
import re

# Estandarizar espacios y puntuación
def standardize_text(text):
    text = re.sub(r'\s+', ' ', text)  # Eliminar espacios duplicados
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar signos de puntuación
    return text.strip()

df_f3['texto'] = df_f3['texto'].apply(standardize_text)

# Mostrar los primeros registros para verificar
display(df_f3.head())

In [ ]:
import re

def mask_pii(text):
    # Mask emails
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', text)
    # Mask phone numbers (basic pattern, can be improved)
    text = re.sub(r'\b\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}\b', '[TEL]', text)
    # Mask names (very basic, will need improvement with a proper NER model)
    text = re.sub(r'\b[A-Z][a-z]+\s[A-Z][a-z]+\b', '[NOMBRE]', text)
    return text

df_f3['texto'] = df_f3['texto'].apply(mask_pii)

# Mostrar los primeros registros para verificar
display(df_f3.head())

In [ ]:
def split_long_text(text, max_words=30):
    words = text.split()
    if len(words) > max_words:
        # Find a reasonable split point (e.g., a sentence boundary)
        split_index = max_words
        while split_index > 0 and words[split_index] != '.':
            split_index -= 1
        if split_index == 0:
            split_index = max_words  # If no sentence boundary, just split at max_words

        first_part = ' '.join(words[:split_index])
        second_part = ' '.join(words[split_index:])
        return first_part, second_part
    else:
        return text, None

# Apply the splitting function
df_f3['texto_split'] = df_f3['texto'].apply(split_long_text)

# Create new rows for the split texts
new_rows = []
for index, row in df_f3.iterrows():
    text1, text2 = row['texto_split']
    if text2:
        new_row1 = row.copy()
        new_row1['texto'] = text1
        new_rows.append(new_row1)

        new_row2 = row.copy()
        new_row2['texto'] = text2
        new_rows.append(new_row2)
    else:
        new_rows.append(row)

df_f4 = pd.DataFrame(new_rows)

# Mostrar los primeros registros para verificar
display(df_f4.head())